In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ames_housing_dummy')

In [3]:
df.head()

,Unnamed: 0,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS SubClass_20,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
0,0,False,False,False,False,True,False,False,False,False,...,210,62,0,0,0,0,0,5,2010,215000
1,1,False,False,False,False,True,False,False,False,False,...,140,0,0,0,120,0,0,6,2010,105000
2,2,False,False,False,False,True,False,False,False,False,...,393,36,0,0,0,0,12500,6,2010,172000
3,3,False,False,False,False,True,False,False,False,False,...,0,0,0,0,0,0,0,4,2010,244000
4,4,False,False,False,False,False,False,False,False,False,...,212,34,0,0,0,0,0,3,2010,189900


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 264 entries, Unnamed: 0 to SalePrice
dtypes: bool(227), float64(11), int64(26)
memory usage: 1.5 MB


**The label we are trying to predict is the SalePrice column. Separating out the data into X features and y labels**

In [5]:
X = df.drop('SalePrice', axis = 1)

In [6]:
y = df['SalePrice']

**Spliting up X and y into a training set and test set.**

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

**Scaling the X features.**

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
scaler = StandardScaler()

In [11]:
scaler.fit(X_train)

StandardScaler()

In [12]:
X_train = scaler.transform(X_train)

In [13]:
X_test = scaler.transform(X_test)

**Creating Elastic Net Model**

In [14]:
from sklearn.linear_model import ElasticNet

In [15]:
base_model = ElasticNet()

In [16]:
param_grid = {'alpha':[0.5,1,5,25,40,60,80,100], "l1_ratio" : [.1,.2,.5,.8,1]}

In [17]:
from sklearn.model_selection import GridSearchCV 

In [18]:
grid_model = GridSearchCV(base_model,param_grid = param_grid,scoring='neg_mean_squared_error',cv = 5,verbose=1 ) 

In [19]:
grid_model.fit(X_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.395e+11, tolerance: 1.355e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.147e+11, tolerance: 1.308e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.789e+11, tolerance: 1.415e

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.5, 1, 5, 25, 40, 60, 80, 100],
                         'l1_ratio': [0.1, 0.2, 0.5, 0.8, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [20]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

In [21]:
y_pred = grid_model.predict(X_test)

In [22]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
mean_absolute_error(y_test,y_pred)

14218.227839037207

In [24]:
np.sqrt(mean_squared_error(y_test,y_pred))

20619.344861796188

In [25]:
np.mean(df['SalePrice'])

180815.53743589742

In [26]:
import pickle

In [27]:
filename = 'final_model_ames_housing.sav'

In [28]:
pickle.dump(grid_model,open(filename, 'wb'))

In [29]:
load_model = pickle.load(open(filename,'rb'))